In [1]:
import os, re, sys, csv, json, gzip, string, importlib
import pandas as pd
import numpy as np
from collections import defaultdict, Counter

In [2]:
#.tokens and .beforeQA.tokens files

In [3]:
novels = []
for folder in os.scandir('pdnc_output/'):
    if os.path.isdir(folder) and folder.name[0] not in ['.', '_']:
        novels.append(folder.name)
novels = sorted(novels)

In [4]:
print(novels, len(novels))

['ARoomWithAView', 'AgeOfInnocence', 'AliceInWonderland', 'AnneOfGreenGables', 'DaisyMiller', 'Emma', 'HandfulOfDust', 'HowardsEnd', 'NightAndDay', 'NorthangerAbbey', 'Persuasion', 'PrideAndPrejudice', 'SenseAndSensibility', 'SignOfFour', 'TheAwakening', 'TheGambler', 'TheInvisibleMan', 'TheManWhoWasThursday', 'TheMysteriousAffairAtStyles', 'ThePictureOfDorianGray', 'TheSportOfTheGods', 'TheSunAlsoRises'] 22


In [5]:
def read_op_csv(path):
    df = pd.read_csv(path, \
                sep='\t', quoting=3, lineterminator='\n')

    return df

## Oct 4

In [6]:
novel = novels[0]
print(novel)

ARoomWithAView


In [ ]:
read_op_csv

In [7]:
op_files = []
for file in os.scandir(os.path.join('pdnc_output/', novel)):
    if os.path.isfile(file):
        op_files.append(file.name)

In [8]:
print(op_files)

['ARoomWithAView.tokens', 'ARoomWithAView.book', 'ARoomWithAView.supersense', 'ARoomWithAView.beforeQA.tokens', 'ARoomWithAView.quotes', 'ARoomWithAView.entities', 'ARoomWithAView.book.html', 'ARoomWithAView.beforeCoref.refs', 'ARoomWithAView.beforeCoref.entities', 'ARoomWithAView.beforeCoref.tokens', 'ARoomWithAView.beforeCoref.qattrib', 'ARoomWithAView.beforeQA.entities', 'ARoomWithAView.beforeCoref.quotes']


In [7]:
tokdf = read_op_csv(os.path.join('pdnc_output/', novel, novel+'.tokens'))

In [8]:
tokdf

,paragraph_ID,sentence_ID,token_ID_within_sentence,token_ID_within_document,word,lemma,byte_onset,byte_offset,POS_tag,fine_POS_tag,dependency_relation,syntactic_head_ID,event,inQuote
0,0,0,0,0,PART,PART,0,4,PROPN,NNP,dep,18,O,False
1,0,0,1,1,ONE,one,5,8,NUM,CD,nummod,1,O,False
2,1,0,2,2,Chapter,chapter,12,19,NOUN,NN,appos,0,O,False
3,1,0,3,3,I,i,20,21,NOUN,NN,nummod,0,O,False
4,1,0,4,4,:,:,21,22,PUNCT,:,punct,0,O,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84691,2164,5828,15,84691,winter,winter,374649,374655,NOUN,NN,pobj,84690,O,False
84692,2164,5828,16,84692,into,into,374656,374660,ADP,IN,prep,84686,O,False
84693,2164,5828,17,84693,the,the,374661,374664,DET,DT,det,84694,O,False
84694,2164,5828,18,84694,Mediterranean,Mediterranean,374665,374678,PROPN,NNP,pobj,84692,O,False


In [6]:
novel = novels[0]

In [7]:
#tokens

In [9]:
tokdf = read_op_csv(os.path.join('pdnc_output/', novel, novel+'.tokens'))

In [10]:
tokdf

,paragraph_ID,sentence_ID,token_ID_within_sentence,token_ID_within_document,word,lemma,byte_onset,byte_offset,POS_tag,fine_POS_tag,dependency_relation,syntactic_head_ID,event
0,0,0,0,0,PART,PART,0,4,PROPN,NNP,dep,18,O
1,0,0,1,1,ONE,one,5,8,NUM,CD,nummod,1,O
2,1,0,2,2,Chapter,chapter,12,19,NOUN,NN,appos,0,O
3,1,0,3,3,I,i,20,21,NOUN,NN,nummod,0,O
4,1,0,4,4,:,:,21,22,PUNCT,:,punct,0,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...
84691,2164,5828,15,84691,winter,winter,374649,374655,NOUN,NN,pobj,84690,O
84692,2164,5828,16,84692,into,into,374656,374660,ADP,IN,prep,84686,O
84693,2164,5828,17,84693,the,the,374661,374664,DET,DT,det,84694,O
84694,2164,5828,18,84694,Mediterranean,Mediterranean,374665,374678,PROPN,NNP,pobj,84692,O


In [11]:
#ent output

In [16]:
entdf = pd.read_csv(os.path.join('pdnc_output/', novel, novel+'.beforeQA.tokens'), header=None)

In [17]:
entdf

,0,1,2,3
0,5,6,PROP_PER,The Bertolini
1,8,9,PROP_PER,The Signora
2,19,20,PROP_PER,Miss Bartlett
3,28,28,PRON_PER,She
4,30,30,PRON_PER,us
...,...,...,...,...
13725,84656,84656,PROP_PER,Phaethon
13726,84665,84665,PRON_PER,they
13727,84681,84681,PRON_PER,they
13728,84683,84684,NOM_LOC,the river
